In [ ]:
import geopandas
import matplotlib.pyplot as plt

In [ ]:
def intersect_counter(centroids, boundaries):

        """ A function that will count the number of centroids that fall within
            the boundaries of another layer.

            Input:
            ------
            centroids (float): Centroid coordinates of each source layer to
            disaggregate.

            boundaries (GeoDataFrame ?): Boundaries of the target layer.

            Output:
            -------
            boundaries (GeoDataFrame): Boundaries GeoDataFrame containing number
            of intersecting source centroids.
        """

        boundaries["count"] = 0

        for i in range(len(boundaries)):
            inter_count=0
            CD = boundaries.loc[[i]]

            inter_count = sum(centroids.intersects(CD))
#             print(inter_count)
            boundaries.loc[i,"count"] = inter_count

        return boundaries

In [ ]:
parcels = geopandas.read_file('testdata/MN_PLUTO_SUBSET.shp')
source = geopandas.read_file('testdata/nad_subset.shp')
print(parcels.crs)
print(source.crs)

source['centroids'] = source.centroid

source2 = source.copy()
source2['geometry'] = source.centroid

source2

In [ ]:
fig, ax = plt.subplots()
parcels.shape
parcels.plot(color='#B3E5FC', ax=ax)
source2.plot(markersize=.5, ax=ax)

In [115]:
inters = [geopandas.sjoin(parcels.loc[[k]], source2).shape[0] for k in parcels.index]

# for k in parcels.index:
#     x = geopandas.sjoin(parcels.loc[[k]], source2)
#     print(x.shape[0])

In [120]:
inters[1207]

15